In [2]:
# --- Import the Salary Data ---

# Import essential libraries
import pandas as pd              # for working with tabular data
from pathlib import Path          # for handling file paths safely across OS

# Define where the dataset is stored
DATA_PATH = Path('../data')       # path to the data folder
CSV_FILE = DATA_PATH / 'Salaries.csv'  # full path to your CSV file

# Quick confirmation
print('Setup complete! Looking for file at:', CSV_FILE)


Setup complete! Looking for file at: ..\data\Salaries.csv


In [17]:
# --- Load and clean the salary dataset (robust version) ---

class DataLoadError(Exception):
    """Raised when there is an issue loading the salary data file."""
    pass


def load_salary_data(csv_path: Path) -> pd.DataFrame:
    """
    Reads the salary CSV and returns a cleaned pandas DataFrame.
    - Treats common text placeholders as missing values
    - Cleans currency symbols/commas
    - Casts numeric columns to proper numeric types
    - Uses nullable Int64 for integer-like columns
    """

    # 1) Sanity check: the file should exist
    if not csv_path.exists():
        raise DataLoadError(f"⚠️ Data file not found at: {csv_path}")

    # 2) Read CSV with sensible defaults
    try:
        df = pd.read_csv(
            csv_path,
            low_memory=False,                         # better dtype inference
            na_values=["Not Provided", "None", "", "-", "N/A"],  # treat these as NaN
        )
    except Exception as e:
        raise DataLoadError(f"❌ Could not read CSV file. Error: {e}")

    # 3) Tidy up a few text fields (nice-to-have)
    for col in ("EmployeeName", "JobTitle", "Agency"):
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    # 4) Clean + convert numeric columns
    numeric_cols = [
        "BasePay", "OvertimePay", "OtherPay", "Benefits",
        "TotalPay", "TotalPayBenefits"
    ]
    for col in numeric_cols:
        if col in df.columns:
            # remove currency symbols and commas, then coerce to numeric
            df[col] = (
                df[col]
                .astype(str)
                .str.replace(r"[\$,]", "", regex=True)
                .str.strip()
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # 5) Cast integer-like columns to nullable integer (keeps NaN)
    for col in ("Id", "Year"):
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    # 6) Friendly confirmation
    loaded_msg = f"✅ Loaded {len(df):,} rows and {df.shape[1]} columns from '{csv_path.name}'"
    if "BasePay" in df.columns:
        loaded_msg += f" | BasePay missing: {int(df['BasePay'].isna().sum()):,}"
    print(loaded_msg)

    return df


In [18]:
# --- Load the data and preview it ---

df = load_salary_data(CSV_FILE)

# Show the first few rows to confirm it loaded correctly
print('Showing the first 5 rows:')
df.head()

✅ Loaded 312,882 rows and 9 columns from 'Salaries.csv' | BasePay missing: 606
Showing the first 5 rows:


,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,NaN,567595.43,567595.43,2011
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,NaN,335279.91,335279.91,2011
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,NaN,332343.61,332343.61,2011
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,NaN,326373.19,326373.19,2011


In [22]:
# --- Create a function to return employee details by name ---

class EmployeeNotFoundError(Exception):
    """Raised when the employee name is not found."""
    pass

def employee_details(name: str, data: pd.DataFrame) -> pd.DataFrame:
    """
    Look up an employee by name (case-insensitive) and return their rows.
    If multiple rows exist (e.g., different years), return them all.
    """

    # 1) Validate the input
    if not isinstance(name, str) or not name.strip():
        raise ValueError("Please provide a non-empty employee name (string).")

    # 2) Case-insensitive match on the EmployeeName column
    mask = data["EmployeeName"].str.lower() == name.strip().lower()
    result = data.loc[mask]

    # 3) If no match, raise a clear error
    if result.empty:
        raise EmployeeNotFoundError(f"No record found for: {name}")

    # 4) Return a tidy set of columns (keeps output readable)
    cols = [
        "Id", "EmployeeName", "JobTitle",
        "BasePay", "OvertimePay", "OtherPay", "Benefits",
        "TotalPay", "TotalPayBenefits",
        "Year", "Agency"
    ]
    return result[[c for c in cols if c in result.columns]]


In [24]:
# --- Process data with dictionary ---

# Convert a few columns into a dictionary structure for quick lookups
salary_dict = df[["EmployeeName", "JobTitle", "TotalPay"]].head(10).to_dict(orient="records")

# Example: print first 5 entries to verify
for record in salary_dict[:5]:
    print(record)


{'EmployeeName': 'NATHANIEL FORD', 'JobTitle': 'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY', 'TotalPay': 567595.43}
{'EmployeeName': 'GARY JIMENEZ', 'JobTitle': 'CAPTAIN III (POLICE DEPARTMENT)', 'TotalPay': 538909.28}
{'EmployeeName': 'ALBERT PARDINI', 'JobTitle': 'CAPTAIN III (POLICE DEPARTMENT)', 'TotalPay': 335279.91}
{'EmployeeName': 'CHRISTOPHER CHONG', 'JobTitle': 'WIRE ROPE CABLE MAINTENANCE MECHANIC', 'TotalPay': 332343.61}
{'EmployeeName': 'PATRICK GARDNER', 'JobTitle': 'DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)', 'TotalPay': 326373.19}


In [25]:
# --- Export one employee's rows to CSV and zip the folder ---

from zipfile import ZipFile, ZIP_DEFLATED
from pathlib import Path

OUTPUT_ROOT = Path("../output")
PROFILE_DIR = OUTPUT_ROOT / "Employee Profile"
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)
PROFILE_DIR.mkdir(parents=True, exist_ok=True)

def safe_filename(text: str) -> str:
    """Make a simple, filesystem-friendly filename."""
    return "".join(ch for ch in text.strip().replace(" ", "_") if ch.isalnum() or ch in {"_", "-"})

def export_employee_profile(name: str, data: pd.DataFrame) -> Path:
    """
    1) Fetch the employee rows using employee_details()
    2) Save them as a CSV in output/Employee Profile/
    3) Create/overwrite output/Employee_Profile.zip containing that CSV
    Returns the path to the zip file.
    """
    # 1) Get the rows (raises EmployeeNotFoundError if none)
    rows = employee_details(name, data)

    # 2) Save CSV (filename includes the name)
    csv_name = f"{safe_filename(name)}_profile.csv"
    csv_path = PROFILE_DIR / csv_name
    rows.to_csv(csv_path, index=False)

    # 3) Zip it (overwrite each time to keep things clean)
    zip_path = OUTPUT_ROOT / "Employee_Profile.zip"
    with ZipFile(zip_path, mode="w", compression=ZIP_DEFLATED) as zf:
        # Write the CSV into the zip under "Employee Profile/<file>"
        zf.write(csv_path, arcname=f"Employee Profile/{csv_name}")

    print(f"✅ Exported: {csv_path}")
    print(f"✅ Created zip: {zip_path}")
    return zip_path


In [26]:
# --- Generate the Employee_Profile.zip once ---

try:
    # Pick a real name from your dataset:
    target_name = "NATHANIEL FORD"   # ← replace with a valid name from df

    zip_path = export_employee_profile(target_name, df)
    print("Zip exists?", zip_path.exists())
except EmployeeNotFoundError as e:
    print("Not found:", e)
except Exception as e:
    print("Unexpected error:", e)


✅ Exported: ..\output\Employee Profile\NATHANIEL_FORD_profile.csv
✅ Created zip: ..\output\Employee_Profile.zip
Zip exists? True
